<div style="text-align:right;color:blue">version id: __VERSION_ID__</div>

# Writing custom modules
In the last semester we have seen how we can use functions to implement algorithms that we want to execute repeatedly. However, a function will only do exactly one thing. What if we want to collect several functions that do related things? For example, we might want to write a set of functions for working with rational numbers and collect these in a function library that be used in several applications. In this notebook we will look at how related functions can be collected in a Python module which can then easily be reused later.

<hr style="height: 2px">

### What you will learn
In this notebook we will cover the following topics:

* working with rational numbers in Python
* Python modules
* Documentation and testing modules
* Python scripts

<hr style="height: 2px">

*&#169; Eike Mueller, University of Bath 2019-2024. These lecture notes are copyright of Eike Mueller, University of Bath. They are provided exclusively for educational purposes at the University and are to be downloaded or copied for your private study only. Further distribution, e.g. by upload to external repositories, is prohibited.*

## Rational numbers
Imagine that we want to write a set of Python functions for working with rational numbers. A rational number $q=\frac{a}{b}$ is defined by its numerator $a$ and its denominator $b$; both $a$ and $b\ne 0$ are integer numbers. The set of all rational numbers is called $\mathbb{Q}$. To make the representation in terms of two integers $a$ and $b$ unique, we also assume that

* the denominator is positive ($b>1$)
* the numerator $a$ and the denominator $b$ have no common factors larger than 1. In other words, their largest common divisor (gcd) is $\text{gcd}(a,b)=1$. To compute the gcd of two numbers we can use the `gcd()` function from the `math` module:

In [ ]:
import math
math.gcd(45,100)

The rules for adding and multiplying two rational numbers $q=\frac{a}{b}\in\mathbb{Q}$ and $r=\frac{c}{d}\in\mathbb{Q}$ are:

$$
q\cdot r = \frac{a\cdot c}{b\cdot d}\qquad\text{and}\qquad q + r = \frac{a\cdot d+b\cdot c}{b\cdot d}
\qquad(\dagger)
$$

### Python implementation
In Python, we can represent a rational number $q=\frac{a}{b}$ as a list $[a,b]$ with two entries $a\in \mathbb{Z}$, $b\in \mathbb{N}$, $b>0$. This immediately leads to the following two functions which implement the sum and the product of two rational numbers. 

```Python
def mul(p,q):
    '''Multiply two rational numbers

    Input:
      * rational number p = [a,b] (= a/b)
      * rational number q = [c,d] (= c/d)

    Output:
      * rational number r = [e,f] (= e/f) with r = p*q
    '''
    num_p, denom_p = p
    num_q, denom_q = q
    num_r = num_p*num_q
    denom_r = denom_p*denom_q
    return _simplify_rational([num_r,denom_r])

def add(p,q):
    '''Add two rational numbers

    Input:
      * rational number p = [a,b] (= a/b)
      * rational number q = [c,d] (= c/d)

    Output:
      * rational number r = [e,f] (=e/f) with r = p + q
    '''
    num_p, denom_p = p
    num_q, denom_q = q
    num_r = num_p*denom_q + num_q*denom_p
    denom_r = denom_p*denom_q
    return _simplify_rational([num_r,denom_r])
```

The final line in each function calls a function `_simplify_rational()` which ensures that the gcd of the returned numerator and denominator is 1:

```Python
def _simplify_rational(q):
    '''Cancel common integer factors in numerator and denominator

    Input:
     * rational number q = [a, b] (= a/b)

    Output:
     * Simplified rational number q = (a'/b') with a' = a/g, b' = b/g where
       g = gcd(a,b)
    '''
    num, denom = q
    g = math.gcd(num,denom)
    return [num//g,denom//g]
```

## Python modules
If we implement these functions in a Python notebook, we can only ever use them in this particular notebook, and we have to copy and paste them if we want to use them somewhere else. This is clearly bad style. There is, however, a better solution: We can collect the three functions in a python module which can be loaded into any notebook. For this, we simply save them in a file called `rational.py`. If you go back to the folder containing the present notebook, you will find this file there. Open it now and have a look at the code.

Using the module is very easy. All we need to do is import it with the `import` statement, just as we would import for example the numpy library. We can then use any functions from the module by preceding them with the name of this module, i.e. `rational.`. Let's try this out:

In [ ]:
import rational
p = [1,3]
q = [2,5]
p_plus_q = rational.add(p,q)
p_times_q = rational.mul(p,q)
print ('p + q = ',p_plus_q)
print ('p * q = ',p_times_q)

Note that since the function calls are preceded by the module name, two different modules can contain the same function. Just as we usually import the numpy library such that we can use it with the shorthand `np`, it is also possible to re-name a custom module when importing it (this can be useful if the module name is very long):

In [ ]:
import rational as ra
p = [1,3]
q = [2,5]
p_plus_q = ra.add(p,q)
p_times_q = ra.mul(p,q)

The only caveat is that Python is only able to import modules which are stored in the current directory or in one of the directories on the [module search path](https://docs.python.org/3/tutorial/modules.html#the-module-search-path). For this course the module search path has been set up correctly.

If you look at the file `rational.py` you will see that it contains three additional functions:

* `to_str(q)` takes a rational number `q` (given as a list with two entries `[a,b]`) and converts it to a well-formatted string of the form `a / b` which can be printed out. If the denominator is one, only the numerator is printed.
* `add_int(n,q)` can be used to add an integer $n$ and a rational number $q=\frac{a}{b}$
* `mul_int(n,q)` can be used to multiply an integer $n$ and a rational number $q=\frac{a}{b}$

We now have a handy little library for doing some non-trivial calculations. Here are some examples:

In [ ]:
p = [13,24]
q = [-12,7]
n = 5
print(rational.to_str(p),' + ',rational.to_str(q),' = ',rational.to_str(rational.add(p,q)))
print(rational.to_str(p),' * ',rational.to_str(q),' = ',rational.to_str(rational.mul(p,q)))
print(str(n),' + ',rational.to_str(p),' = ',rational.to_str(rational.add_int(n,q)))
print(str(n),' * ',rational.to_str(p),' = ',rational.to_str(rational.mul_int(n,q)))

One thing that is still a little awkward about this is that whereas we can write `t = n + m` if `n` and `m` are integers, we can not write `r = p + q` is `p` and `q` are rational numbers represented by lists. In fact, if we try this we will get the following result:

In [ ]:
p = [13,24]
q = [-12,7]
r = p + q
print(r)

This is clearly not what we wanted! The reason is of course that Python does not know that the lists `p` and `q` represent rational numbers. It is also quite awkward that we have to use `rational.add()` for adding two rational numbers but `rational.add_int()` if we want to add a rational number and an integer. In one of the later lectures we will see how these problems can be addressed in an elegant way by using object oriented programming and operator overloading.

## Documenting and testing modules
### Module documentation
If we look at `rational.py` we find that it starts with a doc-string in triple inverted commas:
```Python
'''Algebra of rational numbers

This module provides functions for multiplying and adding rational numbers.
Rational numbers are represented as pairs of integers. For example, the rational
number a / b is stored as [a, b].
'''
```
As it is good practice to document functions, modules should also be documented. The doc-string at the beginning of a module is split into a one-line summary, followed by a more detailled description. If we call the Python `help()` function with the module name, the documentation of the module will be printed out:

In [ ]:
help(rational)

As we can see, this includes the docstring for the module as well as documentation of all functions defined in the module. There is one exception, however: functions starting with an underscore, such as `_simplify_rational()` are not included. These are typically auxilliary functions that are 'local' to the module: while they might be used by other functions in the module, they are hidden in the documentation since they are not meant to be called by any user of the module (However, Python is quite relaxed about this and does not stop us from doing this: it is perfectly possible to call `rational._simplify_rational(q)` after importing the module).

### Testing with pytest
As with individual functions, we should test whole modules. For this, we collect a set of tests in another Python file called `test_rational.py`. Have a look at this file, which can be found in the same directory as this script.

To run all tests in a particular file we can use the `!pytest` command:

In [ ]:
!pytest test_rational.py

The output can be made a bit more verbose by adding the `-v` flag:

In [ ]:
!pytest -v test_rational.py

When developing a new module it is a good idea to re-run these tests after any changes to the code.

## Standalone Python scripts
Developing longer, more complex code in Jupyter notebooks can be quite cumbersome: if the code is split over different cells, we need to remember to execute these cells in the correct order. For this reason it sometimes makes sense to store code as a separate Python script. For this, we can simply save the code in a file with the `.py` extension. Consider the following code, which computes the sum 

$$
1 + \frac{1}{2} + \frac{1}{4} + \frac{1}{8} + \dots + \frac{1}{2^{15}}
$$

```Python
q = [1,1]
for k in range(1,16):
    q = rational.add(q,[1,2**k])
print (q)
```

You can find this code in the script `sum_rational.py` in the same directory as this notebook. Open this file now and have a look at its contents. To execute the code, we simply call pass its contents to the Python interpreter with 

In [ ]:
!python sum_rational.py

## Check your understanding
Answers to the following questions can be found at the very end of this notebook.

**Q1**: Which of the following statements is correct:

1. A python script stored in the file `my_script.py` can be imported with
```Python
!python my_script.py
```
2. A Python module contains several functions that are logically related
3. Python modules should be stored in files ending in `.py`
4. The documentation of a custom Python module can be displayed with the `help()` function
5. All of the above
6. None of the above

**Q2**: The file `linear_algebra.py` contains the following function for computing the cross-product of two three-dimensional vectors, which are represented as lists:

```Python
def cross_product(u,v):
    w = [u[1]*v[2]-u[2]*v[1],u[2]*v[0]-u[0]*v[2],u[0]*v[1]-u[1]*v[0]]
    return w
```

How can you use this function in your code to compute the cross-product of the two vectors $\begin{pmatrix}1\\2\\3\end{pmatrix}$ and $\begin{pmatrix}4\\5\\6\end{pmatrix}$?

1. ```Python
import linear_algebra
linear_algebra.cross_product([1,2,3],[4,5,6])
```

2. ```Python
import linear_algebra as la
linear_algebra.cross_product([1,2,3],[4,5,6])
```

3. ```Python
import linear_algebra as la
cross_product([1,2,3],[4,5,6])
```

4. All of the above

5. None of the above

$$
{}^{}
\\[20cm]
{}^{}
$$

## Answers to "Check your Understanding" questions
**Q1**: answer #5, **Q2**: answer #1